In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [3]:
df = pd.read_csv('bangkok_traffy.csv')
print(df.shape)
df.head()

(787026, 16)


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


In [4]:
weather_df = pd.read_csv('data/open-meteo-13.81N100.51E7m.csv')
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30696 entries, 0 to 30695
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   time                           30696 non-null  object 
 1   temperature_2m (°C)            30696 non-null  float64
 2   dew_point_2m (°C)              30696 non-null  float64
 3   relative_humidity_2m (%)       30696 non-null  int64  
 4   rain (mm)                      30696 non-null  float64
 5   vapour_pressure_deficit (kPa)  30696 non-null  float64
 6   cloud_cover (%)                30696 non-null  int64  
 7   wind_direction_10m (°)         30696 non-null  int64  
 8   surface_pressure (hPa)         30696 non-null  float64
 9   wind_speed_10m (km/h)          30696 non-null  float64
dtypes: float64(6), int64(3), object(1)
memory usage: 2.3+ MB


In [5]:
traffy = df.drop(['ticket_id', 'organization', 'photo', 'photo_after',
            'address', 'state', 'star', 'count_reopen', 'last_activity'], axis=1)

traffy.dropna(inplace=True)

traffy = traffy[traffy['province'] == 'กรุงเทพมหานคร'].reset_index(drop=True)

In [11]:
traffy.info()
traffy['province'].mode()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   type         651600 non-null  object
 1   comment      651600 non-null  object
 2   coords       651600 non-null  object
 3   subdistrict  651600 non-null  object
 4   district     651600 non-null  object
 5   province     651600 non-null  object
 6   timestamp    651600 non-null  object
dtypes: object(7)
memory usage: 34.8+ MB


0    กรุงเทพมหานคร
Name: province, dtype: object